In [10]:
import os
import numpy as np
from tqdm import tqdm

from PIL import Image

In [4]:
rename = {
    'Baboon_': 'Baboon',
    'Elephant_': 'Elephant',
    'Honey_badger_': 'Honey_badger',
    'Honey_Badger': 'Honey_badger',
    'Mongoose_larger_gray': 'Mongoose_larger_grey',
    'Hippopotamus': 'Hippo'
}

In [7]:
root = '/home/zhmiao/datasets/ecology/Mozambique/'

s1_ori = os.path.join(root, 'SplitLists', 'Mozambique_season_1_all.txt.ori')
s2_ori = os.path.join(root, 'SplitLists', 'Mozambique_season_2_all.txt.ori')

In [8]:
def read_ori_lists(path):

    file_list = []
    label_list = []
    sec_list = []
    
    with open(path, 'r') as f:
        
        for line in tqdm(f):
            
            line = line.replace('  ', ' ')
            
            if 'Honey Badger' in line:
                line = line.replace('Honey Badger', 'Honey_badger')
                
            if 'Mongoose_white tailed' in line:
                line = line.replace('Mongoose_white tailed', 'Mongoose_white_tailed')
                
            line_sp = line.split(' ')
            file = line_sp[0]
            label = line_sp[1]
            sec = line_sp[2]
            
            if file.split('/')[1] in rename.keys():
                file_sp = file.split('/')
                file_sp[1] = rename[file_sp[1]]
                file = '/'.join(file_sp)
                
            file_list.append(file)
            label_list.append(label)
            sec_list.append(float(sec.replace('\n', '')))
            
            #if os.path.exists(os.path.join(root, file)):
            #    file_list.append(file)
            #    label_list.append(label)
            
    file_list = np.array(file_list)
    label_list = np.array(label_list)
    sec_list = np.array(sec_list)
    
    return file_list, label_list, sec_list

In [11]:
file_list_s1, label_list_s1, sec_list_s1 = read_ori_lists(s1_ori)
file_list_s2, label_list_s2, sec_list_s2 = read_ori_lists(s2_ori)

313020it [00:00, 415242.95it/s]
323582it [00:00, 456568.29it/s]


In [ ]:
seas_min_all_count = {1: 70, 2: 25}
seas_min_test_count = {1: 50, 2: 20}

In [29]:
def category_selection(label_list, min_count):

    unique_labels, label_counts = np.unique(label_list, return_counts=True)
    
    cat_sel = [(cat, count)
               for cat, count in zip(unique_labels[label_counts > min_count], label_counts[label_counts > min_count]) 
               if 'nknown' not in cat
               and 'other' not in cat
               and cat not in ['Ghost', 'Human', 'Fire', 'Setup', 'Rodent']]
    
    cat_leftout = [(cat, count)
                   for cat, count in zip(unique_labels, label_counts) 
                   if 'nknown' in cat
                   or 'other' in cat
                   or cat in ['Ghost', 'Human', 'Fire', 'Setup', 'Rodent']
                   or count <= min_count]
    
    cat_sel = sorted(cat_sel, key=lambda x : x[1], reverse=True)
    cat_leftout = sorted(cat_leftout, key=lambda x : x[1], reverse=True)
    
    return cat_sel, cat_leftout

In [36]:
cat_sel_counts_s1, cat_leftout_counts_s1 = category_selection(label_list_s1, min_count=150)
cat_sel_counts_s2, cat_leftout_counts_s2 = category_selection(label_list_s2, min_count=150)

In [38]:
set([e[0] for e in cat_sel_counts_s2]) - set([e[0] for e in cat_sel_counts_s1])

In [39]:
set([e[0] for e in cat_sel_counts_s1]) - set([e[0] for e in cat_sel_counts_s2])

{'Aardvark',
 'Duiker_common',
 'Hare',
 'Honey_badger',
 'Mongoose_bushy_tailed',
 'Mongoose_marsh',
 'Mongoose_slender'}

# 1. Combine Two Seasons